In [4]:
from langchain_huggingface import HuggingFaceEmbeddings

model_name = "sentence-transformers/all-MiniLM-L6-v2"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
embeddings_model = HuggingFaceEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

In [1]:
from dotenv import load_dotenv
import os

# Load the .env file
load_dotenv()

# Now you can access your API key and environment like this
api_key = os.getenv("PINECONE_API_KEY")

In [2]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(
    api_key=api_key
)

if 'chess-moves' not in pc.list_indexes().names():
    pc.create_index(
        name='chess-moves',
        vector_type="dense",
        dimension = 384,
        metric="dotproduct",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )

/home/ec2-user/SageMaker/spring-2025-final-project-project-group-13/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from langchain_pinecone import PineconeVectorStore

In [6]:
index = pc.Index('chess-moves')
vector_store = PineconeVectorStore(index=index, embedding=embeddings_model)

In [ ]:
from langchain_community.vectorstores import FAISS

faiss_store = FAISS.load_local("database/all_games", embeddings_model, allow_dangerous_deserialization=True)
docs = faiss_store.docstore._dict.values()

In [ ]:
# vector_store.add_documents(documents=docs)

In [7]:
vectorstore = PineconeVectorStore.from_existing_index(
    index_name="chess-moves",
    embedding=embeddings_model
)

In [8]:
result = vectorstore.similarity_search(
    "rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1",
    k=2
)

In [9]:
result

[Document(id='152084b3-c5fa-46d1-9115-2e3e45618e26', metadata={'black': 'vinniethepooh', 'blackElo': '2389', 'date': '2019.03.05', 'fen': 'rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1', 'move': 'Nf3', 'result': '1/2-1/2', 'termination': 'Game drawn by agreement', 'url': 'https://www.chess.com/game/live/3509059261', 'white': 'GukeshDommaraju', 'whiteElo': '2309'}, page_content='rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1'),
 Document(id='c746977c-09c6-4418-9018-d9d1d5804243', metadata={'black': 'Tijd_Verdoening', 'blackElo': '2430', 'date': '2019.09.22', 'fen': 'rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1', 'move': 'd4', 'result': '1-0', 'termination': 'GukeshDommaraju won by checkmate', 'url': 'https://www.chess.com/game/live/4054728600', 'white': 'GukeshDommaraju', 'whiteElo': '2457'}, page_content='rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1')]